In [1]:
import numpy as np
import h5py
import os
import csv
import time
from torch import manual_seed
import h5py
from torch.utils.data import random_split
import pandas as pd

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

# ---------------------------------------
# 1. Create a binary classification dataset
# ---------------------------------------
X, y = make_classification(
    n_samples=5000,
    n_features=20,
    n_informative=5,
    n_redundant=2,
    weights=[0.9, 0.1],      # imbalanced dataset
    random_state=42
)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# ---------------------------------------
# 2. Train a classifier
# ---------------------------------------
clf = LogisticRegression(max_iter=2000)
clf.fit(X_train, y_train)

# ---------------------------------------
# 3. Predict probabilities
# ---------------------------------------
y_prob = clf.predict_proba(X_test)[:, 1]  # probability of positive class

# AUROC
auroc = roc_auc_score(y_test, y_prob)

# AUPRC (Average Precision)
auprc = average_precision_score(y_test, y_prob)

# ---------------------------------------
# 5. Print results
# ---------------------------------------
print(f"AUROC: {auroc:.4f}")
print(f"AUPRC: {auprc:.4f}")

# Optional: check baseline PR (positive prevalence)
baseline_auprc = np.mean(y_test)
print(f"Baseline AUPRC (positive prevalence): {baseline_auprc:.4f}")


AUROC: 0.8588
AUPRC: 0.5258
Baseline AUPRC (positive prevalence): 0.1033


In [10]:
y_prob
# y_test

array([0.15291125, 0.0039218 , 0.10668613, ..., 0.30721057, 0.1771515 ,
       0.06811176], shape=(1500,))

In [5]:
X.shape
y.shape

(5000,)

In [9]:
path = "./script/Trained_Models/Code15/ECGTransForm/EVAL/Code15 Test_Folder/Stored_Model_Output.hdf5"
x = {}
with h5py.File(path, "r") as f:
    for key in f.keys():
        x[key] = f[key][()]

In [36]:
x['sample_time_points'][65]

np.float64(5.036664191919192)

In [38]:
x['concordance_store_all_ecg'][13]

np.float64(0.7792775336208247)

In [35]:
x["AUPRC"][65]

np.float64(0.1292698492678316)

In [23]:
for i in range(100):
    if x["AUROC"][i] != -1:
        print(i)

13
26
65
99


In [101]:
x = pd.read_csv("/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/Labels_Code15_mort_032025_pd_8020.csv")

In [ ]:
train_ecg_path = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/Code15_ECG.hdf5"
y = {}

with h5py.File(train_ecg_path, "r") as f:
    tmp = x[x['train_test_split']=="train"]
    ECG_SID_Row  = {SID:i for i,SID in enumerate(f['SID'])} # Which ECG.h5 row corresponds to which SID?
    rows_to_pull = np.array([ECG_SID_Row[SID] for SID in tmp['SID']])
    y['ecg'] = f["ECG"][np.sort(rows_to_pull)]
    # print(f.keys())
    # print(f['ECG'].shape)
    # y["ecg"] = f['ECG'][:] # 20000
    # y['pid'] = f['PID'][:]
    # y['sid'] = f['SID'][:]

In [136]:
from pycox.models import LogisticHazard

tmp = x[x['train_test_split']=="train"]
max_duration = max(tmp['Mort_TTE'])
labtrans = LogisticHazard.label_transform(100)
idx_durations, events = labtrans.fit_transform(np.array([0,max_duration]), np.array([0,1]))

a,b = labtrans.transform(tmp['Mort_TTE'].to_numpy(),tmp['Mort_Event'].astype(int).to_numpy())
tmp['Disc_TTE'] = a

/tmp/ipykernel_3665247/1876571392.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['Disc_TTE'] = a


In [93]:
tmp['Mort_Event'].astype(int).to_numpy()

array([0, 0, 0, ..., 0, 0, 0], shape=(11380,))

In [ ]:
np.array([0.01])

(1,)

In [132]:
z = np.array([1, 2, 3, 4])
z[np.array([False, True, True, False])]

array([2, 3])

In [137]:
labtrans.transform(np.array([7.61, 0.01]), np.array([1, 0]))

(array([99,  0]), array([1., 0.], dtype=float32))

In [4]:
import os
os.path.dirname(os.getcwd())

import torch
torch.cuda.get_rng_state()

tensor([123, 209,  26, 115, 157, 218,  21,   0,   0,   0,   0,   0,   0,   0,
          0,   0], dtype=torch.uint8)

In [66]:
tmp

,Unnamed: 0,index,SID,Age,Is_Male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,PID,Mort_Event,Mort_TTE,normal_ecg,trace_file,train_test_split,Disc_TTE
3,3,355,2660251,42,False,48.175446,False,False,False,False,False,False,160919,False,7.243830,False,exams_part0.hdf5,train,93
5,5,357,1047116,73,True,75.276790,False,False,False,False,False,False,699142,False,2.334245,False,exams_part0.hdf5,train,30
6,6,358,411184,82,False,93.824820,False,False,False,False,False,False,1687284,False,3.723285,False,exams_part0.hdf5,train,48
10,10,362,1278551,82,False,79.989070,False,False,False,False,False,False,1517732,False,1.901368,False,exams_part0.hdf5,train,24
13,13,365,2905463,19,False,27.834717,False,False,False,False,False,False,1493110,False,6.564378,False,exams_part0.hdf5,train,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19992,19992,344088,254925,46,True,52.909176,False,False,False,False,False,False,1090352,False,4.073969,True,exams_part0.hdf5,train,52
19994,19994,344090,433358,27,False,46.071053,False,False,False,False,False,False,649876,False,3.638353,False,exams_part0.hdf5,train,46
19997,19997,344093,1439610,37,False,37.154194,False,False,False,False,False,False,231567,False,1.545204,True,exams_part0.hdf5,train,19
19998,19998,344094,3096455,74,True,76.501396,False,False,False,False,False,False,611873,False,6.589036,False,exams_part0.hdf5,train,84


In [63]:
len(labtrans.cuts)
labtrans.idu

In [8]:
x['Mort_Event']=x['Mort_Event'].values.astype(bool)
x

,Unnamed: 0,index,SID,Age,Is_Male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,PID,Mort_Event,Mort_TTE,normal_ecg,trace_file,train_test_split
0,0,352,3158243,59,False,64.235565,False,False,False,False,False,False,1280570,True,NaN,False,exams_part0.hdf5,NegTTE
1,1,353,1126697,63,False,67.575520,False,False,False,False,False,False,223781,False,2.178080,True,exams_part0.hdf5,test
2,2,354,91288,49,False,55.587772,False,False,False,False,False,False,1546425,True,NaN,False,exams_part0.hdf5,NegTTE
3,3,355,2660251,42,False,48.175446,False,False,False,False,False,False,160919,False,7.243830,False,exams_part0.hdf5,train
4,4,356,4390974,42,True,47.617800,False,False,False,False,False,False,98649,True,NaN,False,exams_part0.hdf5,NegTTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,344091,1133183,46,False,61.897950,False,False,False,False,False,False,161647,True,NaN,True,exams_part0.hdf5,NegTTE
19996,19996,344092,531004,71,True,78.271890,False,False,False,False,False,False,110931,True,NaN,False,exams_part0.hdf5,NegTTE
19997,19997,344093,1439610,37,False,37.154194,False,False,False,False,False,False,231567,False,1.545204,True,exams_part0.hdf5,train
19998,19998,344094,3096455,74,True,76.501396,False,False,False,False,False,False,611873,False,6.589036,False,exams_part0.hdf5,train


In [6]:
train_ecg_path = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/Code15_ECG.hdf5"
train_df = 
# open hdf5
with h5py.File(train_ecg_path, "r") as f:
    
    # Pull Training ECG data
    ECG_SID_Row  = {SID:i for i,SID in enumerate(f['SID'])} # Which ECG.h5 row corresponds to which SID?
    rows_to_pull = np.array([ECG_SID_Row[SID] for SID in train_df['SID']]) # Find the rows for our dataframe
    Data['ECG_train'] = f['ECG'][np.sort(rows_to_pull)] # pull those rows in ascending order (hp5y requirement)
    train_df['ECG_Row_Num'] = rows_to_pull # sort the dataframe in ascending row-pulled order
    train_df = train_df.sort_values(by=["ECG_Row_Num"]).reset_index(drop=True)

exams_part0.hdf5
Keys: <KeysViewHDF5 ['exam_id', 'tracings']>


In [13]:
import h5py
from scipy.signal import resample

code15_ecg_dir = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/Code15_ECG.hdf5"

with h5py.File(code15_ecg_dir, "r") as h:
    # print(h['tracings'][:2].shape)
    # y = resample(h['tracings'][:1], 1000, axis = 1)
    # x.append(y)
    # x = h['SID'][()]
    ECGs = resample(h['ECG'][()], 1000, axis = 1)
    EIDs = h['SID'][()]
    labels = h['PID'][()]


In [14]:
tmp = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/HDF5_DATA/Code15/100hz_Code15_ECG.hdf5"

with h5py.File(tmp, "w") as f:
    f.create_dataset('ECG',       data = ECGs, chunks = (1,1000,12),  compression="gzip")
    f.create_dataset('SID',       data = EIDs)
    f.create_dataset('PID',       data = labels)